In [1]:
from pybtex.database import parse_file
import json
from pprint import pprint
import os

In [2]:
# changes names format to a single string (ex: "Robert Downey Jr.")

def name_filtering(data,test):
    for j in test :
        dataset = []
        if 'author' in test[j].persons:
            str = 'author'
        elif 'editor' in test[j].persons: 
            str = 'editor'
        else: 
            continue
        for i in test[j].persons[str]:
            first = ""
            middle = ""
            last = ""
            line = ""
            all = ""
            if i.first_names:
                all += i.first_names[0]
            if i.middle_names:
                all += " " + i.middle_names[0]
            if i.last_names:
                all += " " + i.last_names[0]
            if i.lineage_names:
                all += " " + i.lineage_names[0]
            dataset.append(all)
        data[j] = {'names': dataset, 'year': test[j].fields['year'], 'title': test[j].fields['title']}
    return data

In [3]:
# creates links and nodes 

def setup(data,ref_list,to_import):
    jfile = {}
    jfile["nodes"] = []
    jfile["links"] = []
    for article in data :
        to_do = False
        for k in data[article]['names']:
            if k in ref_list:
                to_do = True
                break
        if to_do:
            for i in data[article]['names']:
                node = {}
                node["id"] = i
                node["lastColaboration"] = "0"
                if i in ref_list:
                    node["group"] = find_team(i,to_import)
                else:
                    node["group"] = "no team"
                new_node = True
                for _node in jfile["nodes"]:
                    if _node["id"] == i:
                        new_node = False
                if new_node:
                    jfile["nodes"].append(node)
                for j in data[article]['names'] :
                    if (i != j) :
                        new = True
                        for k in jfile["links"]:
                            if (k["source"] == j) & (k["target"] == i) :
                                new = False
                                break
                            elif (k["source"] == i) & (k["target"] == j) :
                                if data[article]['title'] not in k["titles"]:
                                    k['titles'].append(data[article]['title'])
                                    k["nbLinks"] += 1
                                
                                link["lastColaboration"] = max(link["lastColaboration"],data[article]['year'])
                                new = False
                                break
                        if new:
                            link = {}
                            link["source"] = i
                            link["target"] = j
                            link["nbLinks"] = 1
                            link["titles"] = [data[article]['title']]
                            link["lastColaboration"] = data[article]['year']
                            jfile["links"].append(link)
                                                        
                for j in range(len(jfile["nodes"])):
                    if jfile["nodes"][j]["id"] == i:
                        jfile["nodes"][j]["lastColaboration"] = max(jfile["nodes"][j]["lastColaboration"],data[article]['year'])
                        break
                    
    return jfile


In [4]:
def clean_data(data):
    tab = []
    tab.append(['{\\\'{e}}','é'])
    tab.append(['{\\\'{E}}','É'])
    tab.append(['{-}','-'])
    tab.append(['{\\^{o}}','ô'])
    tab.append(['{\\"{e}}','ë'])
    tab.append(['{\\"{u}}','ü'])
    tab.append(['{\\`{e}}','è'])
    tab.append(['{\\`{o}}','ò'])
    tab.append(['{\\"{a}}','ä'])
    tab.append(['{\\ae}','æ'])
    tab.append(['{\\ss}','ß'])
    tab.append(['{\\"{o}}','ö'])
    tab.append(['{\\"{\\i}}','ï'])
    tab.append(['{\\\'{a}}','à'])
    tab.append(['{\\^{\\i}}','î'])
    tab.append(['{\\\'{\\i}}','í'])
    tab.append(['{\\~{n}}','ñ'])
    tab.append(['{\\c{c}}','ç'])
    tab.append(['{\\c{C}}','Ç'])
    tab.append(['{\oe}','œ'])
    tab.append(['{\\`{a}}','à'])
    tab.append(['{\\(\\mathscr{l}\\)}','ℓ'])
    tab.append(['\\emph{delta}','δ'])
    tab.append(['{\\^{a}}','â'])
    tab.append(['{\\&}','&'])
    tab.append(['{\\\'{o}}','ó'])
    tab.append(['{\\(\\mu\\)}','μ'])
    tab.append(['{\\~{a}}','ã'])
    tab.append(['{\\"{O}}','Ö'])
    tab.append(['{\\^{e}}','ê'])
    tab.append(['\\{é\\}','é'])
    
    
    for article in data :
        for pair in tab:
            for i in range(len(data[article]['names'])):
                data[article]['names'][i] = data[article]['names'][i].replace(pair[0],pair[1])
            data[article]['title'] = data[article]['title'].replace(pair[0],pair[1])
        

In [5]:
#save results in json file

def save(jfile):
    with open('data.json', 'w') as f:
        json.dump(jfile, f)

    #pprint(jfile)

In [6]:
def retrieve_files():
    to_import = []
    for x in os.walk(os.getcwd() + "\\bibtex_data"):
        if x[2]:
            team_name = x[0].replace(os.getcwd() + "\\bibtex_data\\", '')
            data = {'team': team_name, 'persons': []}
            for i in x[2]:
                i = i.replace('.bib','')
                data['persons'].append(i.split('_'))
            to_import.append(data)

    ref_list = []
    for i in range(len(to_import)):
        for j in to_import[i]['persons']:
            ref_list += [j[1] + ' ' + j[0]]
    
    return to_import,ref_list

In [7]:
def find_team(name,to_import):
    usable_name = list(reversed(name.split(" ")))
    if len(usable_name) > 2:
        usable_name = [" ".join(reversed(usable_name[0:len(usable_name)-1])),usable_name[len(usable_name)-1]]
    for team in to_import:
        if usable_name in team["persons"]:
            return team["team"]
    return "unknown_team"

In [8]:
filtered_data = {}

to_import,ref_list = retrieve_files()

for team_names in to_import:
    for people in team_names['persons']:
        data = parse_file('bibtex_data/' + team_names['team'] + '/'+ people[0] + '_' + people[1] + '.bib').entries
        filtered_data = name_filtering(filtered_data,data)
        clean_data(filtered_data)
       
jfile = setup(filtered_data,ref_list,to_import)
save(jfile)